In [2]:
import pandas as pd
import sqlite3

**Merging and Joining**

In [ ]:
# Load data from the Chinook database
with sqlite3.connect('data/chinook.db') as con:
    df_customers = pd.read_sql_query('SELECT CustomerId, FirstName, LastName FROM customers', con)
    df_invoices = pd.read_sql_query('SELECT InvoiceId, CustomerId FROM invoices', con)

# Perform an inner join on the CustomerId column
merged_df = pd.merge(df_customers, df_invoices, on='CustomerId', how='inner')

# Count the number of invoices for each customer
customer_invoice_counts = (
    merged_df.groupby(['CustomerId', 'FirstName', 'LastName'])
    .size()
    .reset_index(name='TotalInvoices')
    .sort_values(by='TotalInvoices', ascending=False)
)

# Display the results
print(customer_invoice_counts)

In [ ]:
# Step 1: Load the movie.csv file
file_path = 'data/movie.csv' 
movies = pd.read_csv(file_path)

# Step 2: Create two smaller DataFrames
df_director_color = movies[['director_name', 'color']]
df_director_reviews = movies[['director_name', 'num_critic_for_reviews']]

# Step 3: Perform a left join
left_join = pd.merge(df_director_color, df_director_reviews, on='director_name', how='left')

# Step 4: Perform a full outer join
outer_join = pd.merge(df_director_color, df_director_reviews, on='director_name', how='outer')

# Step 5: Count the number of rows in each resulting DataFrame
print(f"Rows in left join DataFrame: {len(left_join)}")
print(f"Rows in outer join DataFrame: {len(outer_join)}")


**Grouping and Aggregating**

In [ ]:

# Load the Titanic dataset
# Replace 'data/titanic.csv' with the correct path to your Titanic dataset file
titanic = pd.read_excel('data/titanic.xlsx')

# Perform grouped aggregation by Pclass
grouped_results = titanic.groupby('Pclass').agg(
    Average_Age=('Age', 'mean'),      # Calculate average age
    Total_Fare=('Fare', 'sum'),       # Calculate total fare
    Passenger_Count=('PassengerId', 'count')  # Count passengers
).reset_index()

# Display the resulting DataFrame
print(grouped_results)

In [ ]:
# Load the movie data
movies = pd.read_csv('data/movie.csv')

# Perform multi-level grouping by 'color' and 'director_name'
grouped_results = movies.groupby(['color', 'director_name']).agg(
    Total_Num_Critic_Reviews=('num_critic_for_reviews', 'sum'),  # Total critic reviews
    Average_Duration=('duration', 'mean')  # Average duration
).reset_index()

# Display the results
print(grouped_results)

In [ ]:
# Load the flight data
flights = pd.read_parquet('data/flights', columns=['Year', 'Month', 'ArrDelay', 'DepDelay', 'Flights'])

# Drop rows with missing values in relevant columns
flights['ArrDelay'] = pd.to_numeric(flights['ArrDelay'], errors='coerce')
flights = flights[pd.to_numeric(flights['ArrDelay'], errors='coerce').notna()]
cleaned_flights = flights.dropna(subset=['ArrDelay', 'DepDelay', 'Flights'])

# Step 2: Perform nested grouping by 'Year' and 'Month'
grouped_results = cleaned_flights.groupby(['Year', 'Month']).agg(
    Total_Flights=('Flights', 'count'),        # Total number of flights
    Average_Arrival_Delay=('ArrDelay', 'mean'),  # Average arrival delay
    Max_Departure_Delay=('DepDelay', 'max')      # Maximum departure delay
).reset_index()

# Step 3: Display the results
grouped_results.head()

**Applying Functions**


In [ ]:
# Load the Titanic dataset
titanic_df = pd.read_excel('data/titanic.xlsx')

# Define the custom function to classify passengers
def classify_age(age):
    if pd.isna(age):  # Handle missing values
        return "Unknown"
    elif age < 18:
        return "Child"
    else:
        return "Adult"

# Apply the function to the 'Age' column and create a new column 'Age_Group'
titanic_df['Age_Group'] = titanic_df['Age'].apply(classify_age)

# Display the first few rows to verify the new column
print(titanic_df[['Age', 'Age_Group']].head())

In [ ]:

# Load the CSV file
df = pd.read_csv('data/employee.csv')

# Ensure the relevant columns are present
if 'DEPARTMENT' in df.columns and 'BASE_SALARY' in df.columns:
    employees = df[['DEPARTMENT', 'BASE_SALARY']]
else:
    raise ValueError("The required columns 'DEPARTMENT' and 'BASE_SALARY' are not present in the CSV file.")

# Step 3: Normalize the salaries within each department
def normalize_salaries(group):
    # Min-Max Normalization
    group['NORMALIZED_SALARY'] = (group['BASE_SALARY'] - group['BASE_SALARY'].min()) / (group['BASE_SALARY'].max() - group['BASE_SALARY'].min())
    return group

# Apply the normalization within each department
normalized_employees = employees.groupby('DEPARTMENT', group_keys=False).apply(normalize_salaries).reset_index(drop=True)

# Display the normalized data
normalized_employees.head(50)

In [ ]:
# Load the employee data
emp_df = pd.read_csv('data/employee.csv')

# Define a function to normalize salaries within a group
def normalize_salaries(group):
    mean_salary = group.mean()
    std_salary = group.std()
    # Avoid division by zero if all salaries are the same (std = 0)
    if std_salary == 0:
        return 0
    return (group - mean_salary) / std_salary

# Apply the normalization within each department (grouped by DEPARTMENT)
emp_df['Normalized_Salary'] = emp_df.groupby('DEPARTMENT')['BASE_SALARY'].transform(normalize_salaries)

# Display the result
print(emp_df[['BASE_SALARY', 'Normalized_Salary']])

In [ ]:
# Loading the dataset
movie_df = pd.read_csv('data/movie.csv')


# Defining the function
def classify_movie_duration(duration):
    if duration < 60:
        return "Short"
    elif 60 <= duration <= 120:
        return "Medium"
    else:
        return "Long"
    
# Apply the function to the 'duration' column
movie_df['duration_category'] = movie_df['duration'].apply(classify_movie_duration)

# View the result
movie_df.head()

**Using pipe**

In [ ]:
# Load the Titanic dataset
titanic_df = pd.read_excel('data/titanic.xlsx')

# Define the pipeline functions
def filter_survivors(df):
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

def add_fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df

# Create the pipeline
pipeline = (titanic_df.pipe(filter_survivors)
              .pipe(fill_missing_age)
              .pipe(add_fare_per_age))

# Display the result
print(pipeline)

In [ ]:

# Load the flight data
df_flights = pd.read_parquet('data/flights', columns=['DepDelay', 'Scheduled_Duration'])


# Define the pipeline functions
def filter_delays(df):
    return df[df['DepDelay'] > 30]

def add_delay_per_hour(df):
    df['Delay_Per_Hour'] = df['DepDelay'] / df['Scheduled_Duration']
    return df

# Create the pipeline
pipeline = (df.pipe(filter_delays)
                  .pipe(add_delay_per_hour))

# Display the result
print(pipeline)